# How does wildfire impact air quality?

## The goal of this notebook is to build a multiple linear regression model exploring the impact of widlfire on air quality.

#### y = ß0 + ß1X1 + ß2X2 + ... + ßnXn + error, where y is PM2.5 (fine particulate matter) levels, ß0 is a constant, X1 through Xn are the predictors, and ß1 through ßn are the predictors' coefficients.

### Imports

In [179]:
# Needed imports

import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score

### Data Loading

In [180]:
# Load the data
script_dir = os.getcwd()

df = pd.read_csv(f'{script_dir}/air_quality_weather_fires.csv')

df = df.dropna()

df

,Unnamed: 0,date,site_id,latitude,longitude,state_name,county_name,city_name,site_name,PM25,...,fires_within_100km,has_nearby_fire,datetime,month,day_of_week,is_weekend,season,wildfire_season,fire_distance_category,fire_intensity
0,0,2024-01-01,01-073-0023,33.553056,-86.815000,Alabama,Jefferson,Birmingham,North Birmingham,11.55,...,3,1,2024-01-01,1,0,0,winter,0,close,low
1,1,2024-01-01,04-013-9997,33.503833,-112.095767,Arizona,Maricopa,Phoenix,JLG SUPERSITE,85.35,...,0,1,2024-01-01,1,0,0,winter,0,far,low
2,2,2024-01-01,04-019-1028,32.295150,-110.982300,Arizona,Pima,Tucson,CHILDREN'S PARK NCore,16.30,...,0,1,2024-01-01,1,0,0,winter,0,far,low
3,3,2024-01-01,05-119-0007,34.756189,-92.281296,Arkansas,Pulaski,North Little Rock,PARR,5.90,...,0,1,2024-01-01,1,0,0,winter,0,far,low
4,4,2024-01-01,06-001-0011,37.814781,-122.282347,California,Alameda,Oakland,Oakland West,6.90,...,9,1,2024-01-01,1,0,0,winter,0,close,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19797,19797,2024-12-31,49-035-3015,40.777145,-111.945849,Utah,Salt Lake,Salt Lake City,Utah Technical Center,4.50,...,0,1,2024-12-31,12,1,0,winter,0,far,low
19798,19798,2024-12-31,50-021-0002,43.608056,-72.982778,Vermont,Rutland,Rutland,State of Vermont District Court Parking Lot,4.70,...,0,1,2024-12-31,12,1,0,winter,0,far,low
19799,19799,2024-12-31,51-087-0014,37.556520,-77.400270,Virginia,Henrico,East Highland Park,MathScience Innovation Center,5.60,...,5,1,2024-12-31,12,1,0,winter,0,close,low
19800,19800,2024-12-31,53-033-0080,47.568236,-122.308628,Washington,King,Seattle,SEATTLE - BEACON HILL,3.40,...,1,1,2024-12-31,12,1,0,winter,0,moderate,low


In [181]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,19792,19793,19794,19795,19796,19797,19798,19799,19800,19801
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,19792,19793,19794,19795,19796,19797,19798,19799,19800,19801
date,2024-01-01,2024-01-01,2024-01-01,2024-01-01,2024-01-01,2024-01-01,2024-01-01,2024-01-01,2024-01-01,2024-01-01,...,2024-12-31,2024-12-31,2024-12-31,2024-12-31,2024-12-31,2024-12-31,2024-12-31,2024-12-31,2024-12-31,2024-12-31
site_id,01-073-0023,04-013-9997,04-019-1028,05-119-0007,06-001-0011,06-013-0002,06-013-1004,06-019-0011,06-037-1103,06-065-8001,...,48-201-1035,48-201-1039,49-035-2005,49-035-3006,49-035-3010,49-035-3015,50-021-0002,51-087-0014,53-033-0080,56-021-0100
latitude,33.553056,33.503833,32.29515,34.756189,37.814781,37.936013,37.9604,36.78538,34.06659,33.99958,...,29.733726,29.670025,40.598056,40.736389,40.78422,40.777145,43.608056,37.55652,47.568236,41.182227
longitude,-86.815,-112.095767,-110.9823,-92.281296,-122.282347,-122.026154,-122.356811,-119.77321,-118.22688,-117.41601,...,-95.257593,-95.128508,-111.894167,-111.872222,-111.931,-111.945849,-72.982778,-77.40027,-122.308628,-104.778334
state_name,Alabama,Arizona,Arizona,Arkansas,California,California,California,California,California,California,...,Texas,Texas,Utah,Utah,Utah,Utah,Vermont,Virginia,Washington,Wyoming
county_name,Jefferson,Maricopa,Pima,Pulaski,Alameda,Contra Costa,Contra Costa,Fresno,Los Angeles,Riverside,...,Harris,Harris,Salt Lake,Salt Lake,Salt Lake,Salt Lake,Rutland,Henrico,King,Laramie
city_name,Birmingham,Phoenix,Tucson,North Little Rock,Oakland,Concord,San Pablo,Fresno,Los Angeles,Rubidoux,...,Houston,Deer Park,Midvale,Salt Lake City,Salt Lake City,Salt Lake City,Rutland,East Highland Park,Seattle,Cheyenne
site_name,North Birmingham,JLG SUPERSITE,CHILDREN'S PARK NCore,PARR,Oakland West,Concord,San Pablo,Fresno - Garland,Los Angeles-North Main Street,Rubidoux,...,Clinton,Houston Deer Park #2,Copper View,Hawthorne,ROSE PARK,Utah Technical Center,State of Vermont District Court Parking Lot,MathScience Innovation Center,SEATTLE - BEACON HILL,Cheyenne NCore
PM25,11.55,85.35,16.3,5.9,6.9,4.7,12.0,22.15,15.4,18.85,...,15.0,7.9,8.0,2.833333,4.666667,4.5,4.7,5.6,3.4,3.6


In [182]:
# Obtain a list of the columns for ease of typing
df.columns

Index(['Unnamed: 0', 'date', 'site_id', 'latitude', 'longitude', 'state_name',
       'county_name', 'city_name', 'site_name', 'PM25', 'CO', 'O3', 'NO2',
       'SO2', 'AQI_PM25', 'AQI_CO', 'AQI_O3', 'AQI_NO2', 'AQI_SO2', 'AQI',
       'temperature_2m_mean', 'temperature_2m_max', 'temperature_2m_min',
       'relative_humidity_2m_mean', 'wind_speed_10m_mean',
       'wind_direction_10m_dominant', 'precipitation_sum',
       'precipitation_hours', 'et0_fao_evapotranspiration', 'weather_code',
       'distance_to_fire_km', 'fire_brightness', 'fire_frp',
       'fires_within_50km', 'fires_within_100km', 'has_nearby_fire',
       'datetime', 'month', 'day_of_week', 'is_weekend', 'season',
       'wildfire_season', 'fire_distance_category', 'fire_intensity'],
      dtype='object')

### Variable Selection

In [183]:
# List numeric variables
# Only use mean temperature (instead of 'temperature_2m_max', 'temperature_2m_min'), mean wind speed (instead of 'wind_direction_10m_dominant'), and precipitation sum (instead of precipitation_hours') to avoid multicollinearity
X_num = ['latitude', 'longitude', 'temperature_2m_mean', 'relative_humidity_2m_mean', 'et0_fao_evapotranspiration', 'wind_speed_10m_mean', 'precipitation_sum', 'fires_within_50km', 'fires_within_100km', 'distance_to_fire_km', 'fire_brightness', 'fire_frp']

# Convert X_num to numerics, not strings
df[X_num] = df[X_num].apply(pd.to_numeric, errors='coerce')
df = df.dropna(subset=X_num + ['PM25']).reset_index(drop=True)

# Has nearby fire and wildfire season are already binary
# However, has nearby fire is described by the number of fires within 50 and 100 km - remove to avoid multicollinearity 
X_binary = ['wildfire_season']

# List categorical variables
# Site_name, Site_ID, state_name, county_name, city_name, and site name are multicollinear with lat and lon
# Fire distance category is correlated with the distance to fire in km so remove that 
X_cat = ['weather_code', 'season', 'fire_intensity']

# Time data are not used as predictors in our model; do not include those

# Reserve y variable in a separate df
y_df = df[['PM25']].copy() 

# Select binary and categorical X variables (still need to scale numeric variables)
selected_columns = X_binary + X_cat 
X_df = df[selected_columns].reset_index(drop=True)

# Scale our numeric variables
scaler = StandardScaler()
scaled_num_X = scaler.fit_transform(df[X_num])
scaled_X_df = pd.DataFrame(scaled_num_X, columns=X_num)

# Combine scaled_X_df and X_df
X_df = pd.concat([X_df, scaled_X_df], axis=1)

# Use One-Hot_Encoding for categorical variables
X_df_processed = pd.get_dummies(X_df, 
                                 columns=X_cat, 
                                 prefix='dummy', 
                                 drop_first=True)


In [184]:
cols = X_df_processed.columns.tolist()

In [185]:
# Set up X and y 
X = X_df_processed[cols].astype('float64')
X = sm.add_constant(X)
y = y_df[['PM25']].to_numpy(dtype='float64')

In [186]:
# Check for multicollinearity by analyzing the Variance Inflation Factor (VIF)
def vif(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns

    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                            for i in range(len(X.columns))]
    print(vif_data)

vif(X)

                       feature          VIF
0                        const  5083.075584
1              wildfire_season     5.781736
2                     latitude     1.622834
3                    longitude     1.358766
4          temperature_2m_mean     7.279816
5    relative_humidity_2m_mean     2.985879
6   et0_fao_evapotranspiration    10.664454
7          wind_speed_10m_mean     1.280819
8            precipitation_sum     3.302316
9            fires_within_50km     1.775617
10          fires_within_100km     1.799376
11         distance_to_fire_km     1.317161
12             fire_brightness     1.549907
13                    fire_frp     6.755448
14         dummy_Dense drizzle     1.546396
15            dummy_Heavy rain     1.864833
16       dummy_Heavy snow fall     1.217724
17         dummy_Light drizzle     3.062828
18          dummy_Mainly clear     1.642493
19      dummy_Moderate drizzle     2.030214
20         dummy_Moderate rain     3.349460
21    dummy_Moderate snow fall  

In [187]:
# Remove variables with moderate to strong multicollinearity (above 5) to improve model performance

# Fire frp and fire brightness instinctively will be correlated, so remove fire frp as per its VIF above 5
# Wildfire season has a high VIF - likely correlated to season
# The category fire_intensity has high VIF - likely due to multicollinearity with fire brightness - remove

# Important note: high VIFs within a single categorical variable are not necessarily a concern
# They likely indicate a small number of reference category cases due to our relatively small dataset
# We will address this later with bootstrapping
# One change to make is to combine certain weather_code values to increase the number of values in a category (will reduce VIF)


"""
Combine cells into clear, cloudy, rainy, snowy:

Notes:
1 ("Clouds generally dissolving or becoming less developed")
2 ("State of sky on the whole unchanged")
3 ("Clouds generally forming or developing")
51 ("Drizzle, not freezing, continuous - slight at time of observation")
53 ("Drizzle, not freezing, continuous - moderate at time of observation")
55 ("Drizzle, not freezing, continuous - heavy (dense) at time of observation") 
61 ("Rain, not freezing, continuous - slight at time of observation")
63 ("Rain, not freezing, continuous - moderate at time of observation"))
65 ("Rain, not freezing, continuous - heavy at time of observation")  
71 ("Continuous fall of snowflakes - slight at time of observation")
73 ("Continuous fall of snowflakes - moderate at time of observation")
75 ("Continuous fall of snowflakes - heavy at time of observation")

So:

Clear => 1, Clear sky, Mainly clear      
2 => Does not tell us about the actual state of the weather, remove
Cloudy => 3, Overcast, Partly cloudy 
Rainy => 51, 53, 55, 61, 63, 65, Dense drizzle, Heavy rain, Light drizzle, Moderate drizzle, Moderate rain, Slight rain
Snowy => 71, 73, 75, Heavy snow fall, Moderate snow fall, Slight snow fall 
"""

df['weather_code'] = df['weather_code'].replace(['1', 'Clear sky', 'Mainly clear'], 'clear')
df = df[df.weather_code != '2']
df['weather_code'] = df['weather_code'].replace(['3', 'Overcast', 'Partly cloudy'], 'cloudy')
df['weather_code'] = df['weather_code'].replace(['51', '53', '55', '61', '63', '65', 'Dense drizzle', 'Heavy rain', 'Light drizzle', 'Moderate drizzle', 'Moderate rain', 'Slight rain'], 'rainy')
df['weather_code'] = df['weather_code'].replace(['71', '73', '75', 'Heavy snow fall', 'Moderate snow fall', 'Slight snow fall'], 'snowy')

# List numeric variables
X_num = ['latitude', 'longitude', 'relative_humidity_2m_mean', 'wind_speed_10m_mean', 'precipitation_sum', 'fires_within_50km', 'fires_within_100km', 'distance_to_fire_km', 'fire_brightness']

# Convert X_num to numerics, not strings
df[X_num] = df[X_num].apply(pd.to_numeric, errors='coerce')

# List categorical variables
# Site_name, Site_ID, state_name, county_name, city_name, and site name are multicollinear with lat and lon
# Fire distance category is correlated with the distance to fire in km so remove that 
X_cat = ['weather_code', 'season']

# Need to drop NaN values again because of the error coercion in X_num
# Drop NaNs only from columns we actually care about
# We include y ('PM25') in the subset to ensure we have targets for all rows
cols_to_check = X_num + X_cat + ['PM25']
df = df.dropna(subset=cols_to_check)

# Reserve y variable in a separate df
y_df = df[['PM25']].copy().reset_index(drop=True)

# Select categorical X variables (still need to scale numeric variables)
X_df_cat = df[X_cat].reset_index(drop=True)

# Scale our numeric variables
scaler = StandardScaler()
scaled_num_X = scaler.fit_transform(df[X_num])
scaled_X_df = pd.DataFrame(scaled_num_X, columns=X_num).reset_index(drop=True)

# Combine scaled_X_df and X_df
X_df = pd.concat([X_df_cat, scaled_X_df], axis=1)

# Use One-Hot_Encoding for categorical variables
X_df_processed = pd.get_dummies(X_df, 
                                columns=X_cat, 
                                prefix='dummy', 
                                drop_first=True)

cols = X_df_processed.columns.tolist()

# Set up X and y 
X = X_df_processed[cols].astype('float64')
X = sm.add_constant(X)
y = y_df[['PM25']].to_numpy(dtype='float64')

In [188]:
# Retest multicollinearity with VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

                      feature       VIF
0                       const  9.929676
1                    latitude  1.170513
2                   longitude  1.308640
3   relative_humidity_2m_mean  1.557208
4         wind_speed_10m_mean  1.181050
5           precipitation_sum  1.312759
6           fires_within_50km  1.771335
7          fires_within_100km  1.792999
8         distance_to_fire_km  1.303029
9             fire_brightness  1.012077
10               dummy_cloudy  2.447175
11                dummy_rainy  3.123563
12                dummy_snowy  1.608250
13               dummy_spring  1.609261
14               dummy_summer  1.598764
15               dummy_winter  1.656949


In [189]:
# Numeric variables
X_num = ['latitude', 'longitude', 'relative_humidity_2m_mean', 'wind_speed_10m_mean', 'precipitation_sum', 'fires_within_50km', 'fires_within_100km', 'distance_to_fire_km', 'fire_brightness']

# Convert X_num to numerics, not strings
df[X_num] = df[X_num].apply(pd.to_numeric, errors='coerce')

# Categorical variables
X_cat = ['weather_code']

# Need to drop NaN values again because of the error coercion in X_num
# Drop NaNs only from columns we actually care about
# We include y ('PM25') in the subset to ensure we have targets for all rows
cols_to_check = X_num + X_cat + ['PM25']
df = df.dropna(subset=cols_to_check)

# Reserve y variable in a separate df and reset index
y_df = df[['PM25']].copy().reset_index(drop=True)

# Select categorical X variables (still need to scale numeric variables)
X_df_cat = df[X_cat].reset_index(drop=True)

# Scale our numeric variables
scaler = StandardScaler()
scaled_num_X = scaler.fit_transform(df[X_num])
scaled_X_df = pd.DataFrame(scaled_num_X, columns=X_num).reset_index(drop=True)

# Combine scaled_X_df and X_df
X_df = pd.concat([X_df_cat, scaled_X_df], axis=1)

# Use One-Hot_Encoding for categorical variables
X_df_processed = pd.get_dummies(X_df, 
                                columns=X_cat, 
                                prefix='dummy', 
                                drop_first=True)

cols = X_df_processed.columns.tolist()

# Set up X and y 
X = X_df_processed[cols].astype('float64')
X = sm.add_constant(X)
y = y_df[['PM25']].to_numpy(dtype='float64')

In [190]:
# Check VIF one last time
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

                      feature       VIF
0                       const  8.085519
1                    latitude  1.168973
2                   longitude  1.302521
3   relative_humidity_2m_mean  1.499641
4         wind_speed_10m_mean  1.157817
5           precipitation_sum  1.311807
6           fires_within_50km  1.771241
7          fires_within_100km  1.788977
8         distance_to_fire_km  1.298851
9             fire_brightness  1.009750
10               dummy_cloudy  2.400590
11                dummy_rainy  2.994046
12                dummy_snowy  1.536072


### Modeling and  Bootstrapping

In [191]:
# Make X and y pd dfs
X = pd.DataFrame(X)
y = pd.DataFrame(y)
y = y.rename(columns={0:'PM25'})

In [192]:
# Make a baseline model
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.30, random_state = 42) 

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.50, random_state = 42) 

model1 = sm.OLS(y_train, X_train).fit()

print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                   PM25   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     166.1
Date:                Fri, 12 Dec 2025   Prob (F-statistic):               0.00
Time:                        01:59:58   Log-Likelihood:                -41390.
No. Observations:               13827   AIC:                         8.281e+04
Df Residuals:                   13814   BIC:                         8.290e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [193]:
# Bootstrap entire process of making a model

def boot(X, y, S):
    
    # Create an array to store the coefficients for each model
    coef_store = np.zeros((S, X.shape[1]))

    n = len(X)

    # Divide each dataset into a test/train split
    for dataset in range(S):

        # Sample indices with replacement
        idx = np.random.choice(n, n, replace=True)

        # Bootstrap samples
        X_b = X.iloc[idx]
        y_b = y.iloc[idx]

        # Fit the model on the training data
        boot_model1 = sm.OLS(y_b,X_b).fit()

        # Store the coefficients
        coef_store[dataset, :] = boot_model1.params

    # Store results as a dataframe
    boot_results = pd.DataFrame(coef_store, columns=X.columns)

    return boot_results.mean(), boot_results.std()

# Bootstrap
mean, std = boot(X_train, y_train, 1000)

print(f"\nBootstrap Coefficient Means: \n{mean} \n\nBootstrap Coefficient Standard Errors: \n{std}")


Bootstrap Coefficient Means: 
const                        7.888663
latitude                    -0.819544
longitude                    0.018936
relative_humidity_2m_mean    0.591552
wind_speed_10m_mean         -1.000857
precipitation_sum           -0.457912
fires_within_50km           -0.029195
fires_within_100km           0.359336
distance_to_fire_km         -0.573169
fire_brightness             -0.120880
dummy_cloudy                -0.007772
dummy_rainy                 -0.958436
dummy_snowy                 -1.485989
dtype: float64 

Bootstrap Coefficient Standard Errors: 
const                        0.120845
latitude                     0.047723
longitude                    0.046488
relative_humidity_2m_mean    0.061955
wind_speed_10m_mean          0.044888
precipitation_sum            0.034836
fires_within_50km            0.207538
fires_within_100km           0.137287
distance_to_fire_km          0.035008
fire_brightness              0.040249
dummy_cloudy                 0.132129


In [194]:
# Compare the bootstrapped coefficients' standard errors to the OLS standard errors
# If they are similar: OLS assumptions are met, traditional inference is reliable
# If they differ greatly: OLS assumptions may be violated; trust bootstrap SEs for inference
# Large standard errors (regardless of method) suggest high uncertainty in coefficient estimates

# OLS standard errors (from summary)
ols_se = model1.bse

# Compare them
comparison = pd.DataFrame({
    'Bootstrap SE': std,
    'OLS SE': ols_se,
    'Abs Val of Diff': np.abs(std - ols_se)
})

print(comparison)

                           Bootstrap SE    OLS SE  Abs Val of Diff
const                          0.120845  0.117838         0.003006
latitude                       0.047723  0.044412         0.003311
longitude                      0.046488  0.046841         0.000353
relative_humidity_2m_mean      0.061955  0.050418         0.011537
wind_speed_10m_mean            0.044888  0.044461         0.000426
precipitation_sum              0.034836  0.046784         0.011948
fires_within_50km              0.207538  0.048804         0.158734
fires_within_100km             0.137287  0.052334         0.084953
distance_to_fire_km            0.035008  0.046020         0.011012
fire_brightness                0.040249  0.041347         0.001098
dummy_cloudy                   0.132129  0.130744         0.001385
dummy_rainy                    0.152917  0.146028         0.006889
dummy_snowy                    0.207407  0.222686         0.015278


They are similar: OLS assumptions are met, traditional inference is reliable

### Residual Examination

In [195]:
df["fitted_vals"]=model1.fittedvalues
df["residuals"]=model1.resid

In [196]:
fig=px.scatter(df, x="fitted_vals", y="residuals")
fig.add_hline(y=0, line_dash="dash", line_color="red")
fig.update_layout(font=dict(size=24, family="Cuprum"))
fig.show() 

As our fitted values for PM25 increase, the spread of residuals increases.

More of our residuals are positive, and some rise above 80.

This, combined with the low R-squared and adjusted R-squared, leads us to conclude that our model is not very accurate at predicting, despite the fact that all predictors aside from longitude are highly significant.

### Comparing Train and Validate Set

In [197]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

rows = []

yhat_tr = model1.predict(X_train)
yhat_val = model1.predict(X_test)

rows.append({
    "Train RMSE": rmse(y_train, yhat_tr),
    "Validate RMSE": rmse(y_test, yhat_val),
    "Train R2": r2_score(y_train, yhat_tr),
    "Validate R2": r2_score(y_test, yhat_val),
})

results = pd.DataFrame(rows).sort_values("Validate RMSE")
print(results.round(4))

   Train RMSE  Validate RMSE  Train R2  Validate R2
0      4.8282         5.0296    0.1261       0.1144


Our train and validate RMSE and R-squared values do not differ significantly. This is a sign that our model adapts well to new data and is robust.

### Ridge Regularization

In [198]:
# Create Ridge regression models with various alphas to find the best one
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
best_alpha = None
best_val_mse = float('inf')

for alpha in alphas:
    # Train on training set
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train, y_train)
    
    # Evaluate on validation set
    y_pred_val = ridge_model.predict(X_val)
    val_mse = mean_squared_error(y_val, y_pred_val)

    print(f"Alpha: {alpha}, Validation MSE: {val_mse}")
    
    if val_mse < best_val_mse:
        best_val_mse = val_mse
        best_alpha = alpha

print(f"\nBest alpha: {best_alpha}")

Alpha: 0.01, Validation MSE: 22.21136862745971
Alpha: 0.1, Validation MSE: 22.211391345345415
Alpha: 1.0, Validation MSE: 22.21161844156152
Alpha: 10.0, Validation MSE: 22.213880060657917
Alpha: 100.0, Validation MSE: 22.23495657104522

Best alpha: 0.01


A note that the validation MSEs for all alphas are very similar. Still use the best one, of course.

In [199]:
# Evaluate on validate set
final_val_model = Ridge(alpha=best_alpha)
final_val_model.fit(X_val, y_val)

y_pred_val = final_val_model.predict(X_val)
val_mse = mean_squared_error(y_val, y_pred_val)

print(f"Final Validate MSE: {val_mse}")
print(f"Final Validate RMSE: {np.sqrt(val_mse)}")

Final Validate MSE: 21.991198148801722
Final Validate RMSE: 4.6894773854665


The ridge model performs better than the OLS model without ridge, as it has a lower RMSE value (4.689) than the OLS validate RMSE (5.0296). Try lasso as well.

### Lasso Regularization

In [200]:
# Create Lasso models with various alphas to find the best one
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
best_alpha = None
best_val_mse = float('inf')

# Use the train and validation data to find the best alpha
for alpha in alphas:
    # Train on training set
    lasso_model = Lasso(alpha=alpha, fit_intercept=True) 
    lasso_model.fit(X_train, y_train)
    
    # Evaluate on validation set
    y_pred_val = lasso_model.predict(X_val)
    val_mse = mean_squared_error(y_val, y_pred_val)

    print(f"Alpha: {alpha}, Validation MSE: {val_mse}")
    
    if val_mse < best_val_mse:
        best_val_mse = val_mse
        best_alpha = alpha

print(f"\nBest alpha: {best_alpha}")

Alpha: 0.01, Validation MSE: 22.238416146055247
Alpha: 0.1, Validation MSE: 22.554921675315757
Alpha: 1.0, Validation MSE: 24.814444634359653
Alpha: 10.0, Validation MSE: 25.697989115640475
Alpha: 100.0, Validation MSE: 25.697989115640475

Best alpha: 0.01


In [201]:
# Evaluate on validate set
final_val_model = Lasso(alpha=best_alpha)
final_val_model.fit(X_val, y_val)

y_pred_val = final_val_model.predict(X_val)
val_mse = mean_squared_error(y_val, y_pred_val)

print(f"Final Validate MSE: {val_mse}")
print(f"Final Validate RMSE: {np.sqrt(val_mse)}")

Final Validate MSE: 22.003444570095517
Final Validate RMSE: 4.690782937857551


The lasso model also performs better than the OLS model with and without ridge, as it has a lower RMSE value (4.6907) than the OLS validated RMSE (5.0296) or the OLS model with ridge regularization (4.689).

### Removing Coefficients with Lasso

In [202]:
coeffs = final_val_model.coef_

remove = np.array(X_val.columns)[coeffs==0]

keep1 = np.array(X_val.columns)[coeffs>0]

keep2 = np.array(X_val.columns)[coeffs<0]

keep = np.concatenate((keep1, keep2))

print(remove)

print(keep)

['const' 'longitude']
['relative_humidity_2m_mean' 'fires_within_100km' 'latitude'
 'wind_speed_10m_mean' 'precipitation_sum' 'fires_within_50km'
 'distance_to_fire_km' 'fire_brightness' 'dummy_cloudy' 'dummy_rainy'
 'dummy_snowy']


Variables to keep are 'relative_humidity_2m_mean' 'fires_within_100km', 'latitude' 'wind_speed_10m_mean' 'precipitation_sum' 'fires_within_50km'
 'distance_to_fire_km' 'fire_brightness' 'dummy_cloudy' 'dummy_rainy' and 
 'dummy_snowy'.

### Final Model Testing - OLS with Coefficients Removed by Lasso

In [203]:
# Make a final model using the test data but remove the coefficients shrunken to zero by lasso

X_test = X_test[keep]

model_final_test = sm.OLS(y_test, X_test).fit()

print(model_final_test.summary())

                                 OLS Regression Results                                
Dep. Variable:                   PM25   R-squared (uncentered):                   0.611
Model:                            OLS   Adj. R-squared (uncentered):              0.610
Method:                 Least Squares   F-statistic:                              422.3
Date:                Fri, 12 Dec 2025   Prob (F-statistic):                        0.00
Time:                        02:00:30   Log-Likelihood:                         -9382.6
No. Observations:                2963   AIC:                                  1.879e+04
Df Residuals:                    2952   BIC:                                  1.885e+04
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

Now all our predictors are statistically signficant and our r-squared and adjusted r-squared values are much higher. Over 60% of the variation in the data are explained by our predictors.

In [206]:
df["fitted_vals"]=model_final_test.fittedvalues
df["residuals"]=model_final_test.resid

fig=px.scatter(df, x="fitted_vals", y="residuals")
fig.add_hline(y=0, line_dash="dash", line_color="red")
fig.update_layout(font=dict(size=24, family="Cuprum"))
fig.show() 

### Conclusions

The residual plot for the model fitted to the tested model is different from the one previously seen; this is because of the lasso regularization that was performed. Now residuals tend positive, but most are close to zero. Their patterns do not change as the fitted values change. This is indicative of a good model.

In [207]:
np.max(y_test['PM25']) - np.min(y_test['PM25'])

87.4

In [208]:
y_pred_test = model_final_test.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print(f"Final Validate MSE: {test_mse}")
print(f"Final Validate RMSE: {np.sqrt(test_mse)}")

Final Validate MSE: 32.96107375329425
Final Validate RMSE: 5.741173551922485


An RMSE of 5.741 on a scale of 87.4 means that our predictions are off by about 6.5% of the range on average. This means that our predictions are usually reasonable.